**Fabio Zanarello**   
April 2020  
Sanger Institute, Cambridge

In [1]:
info = '../data/original_info/MasterCellID_Lookup.csv'
cell = '../data/original_info/model_list_20200204.csv'
img = '../data/images_jpeg/'

In [2]:
import pandas as pd
import math
import glob 

In [3]:
def get_master(filename):
    if ' (' in filename:
        master = filename.split(' ')[0]
        return master
    
    if '-' in filename:
        master = filename.split('-')[0]
        return master
    else:
        master = filename
    
    return master

def convert_to_str(fl):
    if math.isnan(fl):
        string = 'NaN'
    else:
        string = str(int(fl)).split('.')[0]
    return fl

In [4]:
info_df = pd.read_csv(info)
cell_df = pd.read_csv(cell)
files = [x.split('/')[-1].strip('.jpg') for x in glob.glob(img+'*.jpg')]

#dictionary from two columns
SIDM_to_master = pd.Series(info_df['Master Cell ID'].values,index=info_df.SIDM).to_dict()

#master_to_SIDM = pd.Series(info_df.SIDM.values,index=info_df['Master Cell ID']).to_dict()

#dictionary from list, usign function
master_to_img = {get_master(file):file for file in files}

#new column based on value dictionary, then converted to string
cell_df['master_id'] = cell_df["model_id"].map(SIDM_to_master)

#delete nan in master_id and convert id to string
cell_df = cell_df[cell_df.master_id.notna()]
cell_df['master_id'] = cell_df['master_id'].map(str).apply(lambda x: x.split('.')[0])

#master as index and list
cell_df = cell_df.set_index(['master_id'])
available_master = list(cell_df.index)

#rename model_to SIDM
cell_df['SIDM'] = cell_df['model_id']
cell_df = cell_df.drop(columns=['model_id'])

#to stor final rows
final = pd.DataFrame() 

#append new row containing series (master_id, im_id and all info)
for name in files:
    if get_master(name) in available_master:
        to_append = pd.Series([name,get_master(name)], index=['im_id','master_id'])
        new_row = cell_df.loc[get_master(name)].append(to_append)
        final = final.append(new_row, ignore_index=True)
        
#reorder columns
order_header = ['im_id','master_id']+list(cell_df.columns)
final = final[order_header]

In [5]:
final

,im_id,master_id,model_name,synonyms,model_type,growth_properties,doi,pmed,model_treatment,model_comments,...,ethnicity,smoking_status,model_relations_comment,COSMIC_ID,BROAD_ID,CCLE_ID,RRID,HCMI,suppliers,SIDM
0,1877-8278,1877,SU-DHL-8,NaN,Cell Line,Suspension,NaN,NaN,NaN,NaN,...,Unknown,Unknown,NaN,1331038,ACH-000656,SUDHL8_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,CVCL_2207,NaN,DSMZ:ACC-573,SIDM00423
1,1476-8001,1476,SCC-4,NaN,Cell Line,Adherent,NaN,NaN,NaN,NaN,...,Unknown,Unknown,NaN,910904,ACH-000238,SCC4_UPPER_AERODIGESTIVE_TRACT,CVCL_1684,NaN,ATCC:CRL-1624,SIDM01081
2,1068-8758,1068,PCI-4B,PCI-04B;PCI-4M,Cell Line,Unknown,NaN,NaN,NaN,NaN,...,Unknown,Unknown,NaN,1298531,ACH-002297,PCI4B_UPPER_AERODIGESTIVE_TRACT,CVCL_C171,NaN,Unknown:Unknown,SIDM00043
3,2195-8403,2195,HuH-7,JTC-39,Cell Line,Unknown,NaN,NaN,NaN,NaN,...,East Asian,Unknown,NaN,907071,ACH-000480,HUH7_LIVER,CVCL_0336,NaN,JCRB:JCRB0403,SIDM00585
4,162,162,DOK,NaN,Cell Line,Adherent,NaN,NaN,NaN,NaN,...,White,Unknown,NaN,910936,ACH-002234,DOK_UPPER_AERODIGESTIVE_TRACT,CVCL_1180,NaN,ECACC:94122104,SIDM00540
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
731,706-9149,706,HO-1-N-1,Nakata-1,Cell Line,Adherent,NaN,NaN,NaN,NaN,...,Unknown,Unknown,NaN,924111,ACH-002244,HO1N1_UPPER_AERODIGESTIVE_TRACT,CVCL_1284,NaN,JCRB:JCRB0831,SIDM00592
732,153,153,SK-MEL-31,AX-Mel,Cell Line,Adherent,NaN,NaN,NaN,NaN,...,Unknown,Unknown,NaN,909727,ACH-000640,SKMEL31_SKIN,CVCL_0600,NaN,ATCC:HTB-73,SIDM01104
733,1875-8232,1875,NCI-H1048,NaN,Cell Line,Adherent,NaN,NaN,NaN,NaN,...,Unknown,Unknown,NaN,687995,ACH-000866,NCIH1048_LUNG,CVCL_1453,NaN,ATCC:CRL-5853,SIDM00654
734,2138-8023,2138,EC-GI-10,NaN,Cell Line,Adherent,NaN,NaN,NaN,NaN,...,East Asian,Unknown,NaN,753555,ACH-000305,ECGI10_OESOPHAGUS,CVCL_1187,NaN,RIKEN:RCB0774,SIDM00278


In [7]:
final.to_csv('images_info_table.csv', index= False)